## Exploratory Data Analysis of mmDoppler Data

In [1]:
import sys
sys.path.append("../")

import polars as pl
import plotly.express as px

In [2]:
micro_df = pl.read_parquet("../data/micro_df.parquet")
micro_df_subset = pl.read_parquet("../data/micro_df_subset.parquet")

In [ ]:
# Identify the Stationary / Clutter bin
all_doppler = micro_df.select(pl.col("dopplerIdx").list.explode()).to_series()
print(f"Min Doppler: {all_doppler.min()}")
print(f"Max Doppler: {all_doppler.max()}")

px.histogram(all_doppler.to_numpy(), title="Doppler Distribution").show() 

Min Doppler: -64
Max Doppler: 63


#### Check for class imbalance
Activity distribution across all frames

In [3]:
from helper.analysis_utils import ClassDistributionAnalyzer

dist_analyzer = ClassDistributionAnalyzer(micro_df)

# Compute and Display Distribution
dist_df = dist_analyzer.compute_distribution()
print(dist_df)

# Plot Micro activites (Typing, Eating, etc.) -> IDs 10-18
dist_analyzer.plot_distribution(dist_df).show()


shape: (9, 4)
┌──────────┬───────┬────────────────┬────────────┐
│ activity ┆ count ┆ Activity Name  ┆ Percentage │
│ ---      ┆ ---   ┆ ---            ┆ ---        │
│ i64      ┆ u32   ┆ str            ┆ f64        │
╞══════════╪═══════╪════════════════╪════════════╡
│ 10       ┆ 4635  ┆ Phone Typing   ┆ 18.352075  │
│ 12       ┆ 4546  ┆ Sitting        ┆ 17.999683  │
│ 11       ┆ 3600  ┆ Laptop Typing  ┆ 14.254039  │
│ 18       ┆ 2353  ┆ Drinking Water ┆ 9.316598   │
│ 15       ┆ 2287  ┆ Playing Guitar ┆ 9.055274   │
│ 13       ┆ 2153  ┆ Eating         ┆ 8.524707   │
│ 16       ┆ 2116  ┆ Brushing Teeth ┆ 8.378207   │
│ 17       ┆ 2099  ┆ Combing Hair   ┆ 8.310896   │
│ 14       ┆ 1467  ┆ Phone Talking  ┆ 5.808521   │
└──────────┴───────┴────────────────┴────────────┘


#### Doppler Analysis: Range-Doppler Heatmap visualization

In [3]:
from helper.analysis_utils import visualize_activity_grid_inline

# Check if dist_analyzer is defined
if 'dist_analyzer' not in locals():
    from helper.analysis_utils import ClassDistributionAnalyzer
    dist_analyzer = ClassDistributionAnalyzer(micro_df)

fig_grid = visualize_activity_grid_inline(micro_df, dist_analyzer, n_frames=6)
fig_grid.show()


#### Duration analysis
Calculate the average duration of each activity class. Real-time systems usually need a buffer (e.g., "detect typing for 3 seconds before alerting"). Knowing the typical duration helps us set this buffer size.

- [Note] As per the paper, micro-doppler activites were collected at 2 FPS

##### Calculate average duration of Drinking Water Activity

In [4]:
drink_duration_df = micro_df.filter(pl.col("activity") == 18)

In [ ]:
# Check if the data is sorted by datetime
print(f"Is sorted: {drink_duration_df["datetime"].is_sorted()}")

# Sort by datetime if not sorted
if not drink_duration_df["datetime"].is_sorted():
    drink_duration_df = drink_duration_df.sort("datetime")


Is sorted: True


In [26]:
# Show "jumps" for out-of-order data
fig = px.scatter(
    x=range(len(drink_duration_df)), 
    y=drink_duration_df['datetime'].to_list(),
    title="Data Timeline (Index vs Time)",
    labels={'x': 'Row Index', 'y': 'Timestamp'}
)
fig.show()

#[Conclusion] It shows the data was collected only for <2 days. Lacks variability. But atleast it is not out of order.

#### Duration Analysis
Calculating the typical duration of each activity segment to inform real-time buffer sizing.
Assumes a constant sampling rate of 2 FPS for Micro-Activities.


In [5]:
from helper.analysis_utils import ClassDistributionAnalyzer
dist_analyzer = ClassDistributionAnalyzer(micro_df)

duration_stats = dist_analyzer.compute_duration_stats(fps=2.0)
print(duration_stats)


shape: (9, 7)
┌──────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────────┐
│ Activity ┆ num_segments ┆ median_durat ┆ p10_duration ┆ p90_duration ┆ min_duratio ┆ max_duratio │
│ Name     ┆ ---          ┆ ion          ┆ ---          ┆ ---          ┆ n           ┆ n           │
│ ---      ┆ u32          ┆ ---          ┆ f64          ┆ f64          ┆ ---         ┆ ---         │
│ str      ┆              ┆ f64          ┆              ┆              ┆ f64         ┆ f64         │
╞══════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╪═════════════╡
│ Phone    ┆ 1            ┆ 733.5        ┆ 733.5        ┆ 733.5        ┆ 733.5       ┆ 733.5       │
│ Talking  ┆              ┆              ┆              ┆              ┆             ┆             │
│ Drinking ┆ 4            ┆ 155.75       ┆ 128.5        ┆ 736.5        ┆ 128.5       ┆ 736.5       │
│ Water    ┆              ┆              ┆              ┆              ┆     

In [6]:
micro_df.head()

datetime,rangeIdx,dopplerIdx,numDetectedObj,range,peakVal,x_coord,y_coord,doppz,activity,activity_class
datetime[ns],list[i64],list[i64],f64,list[f64],list[i64],list[f64],list[f64],list[list[i64]],i64,str
2023-03-06 22:58:59,"[1, 10, … 63]","[0, 0, … 0]",6.0,"[0.12932, 1.248754, … 7.875]","[6142, 3312, … 1959]","[-0.117188, 0.234375, … 0.0]","[0.0546875, 1.2265625, … 7.875]","[[18095, 16591, … 17600], [18842, 18620, … 17509], … [18550, 18433, … 18326]]",16,"""micro"""
2023-03-06 22:59:00,"[1, 10, … 63]","[0, 0, … 0]",6.0,"[0.12932, 1.249047, … 7.878131]","[6143, 3391, … 1962]","[-0.117188, 0.2734375, … -7.140625]","[0.0546875, 1.21875, … 3.328125]","[[19575, 19419, … 18241], [18494, 18615, … 17190], … [19129, 19013, … 17823]]",16,"""micro"""
2023-03-06 22:59:00,"[1, 42, … 63]","[0, 0, … 0]",7.0,"[0.12932, 5.252098, … 7.878131]","[6140, 1215, … 1960]","[-0.117188, -0.65625, … -7.140625]","[0.0546875, 5.2109375, … 3.328125]","[[15937, 17874, … 16997], [17924, 17291, … 16826], … [18101, 18402, … 17850]]",16,"""micro"""
2023-03-06 22:59:01,"[1, 42, … 63]","[0, 0, … 0]",7.0,"[0.12932, 5.252098, … 7.878131]","[6145, 1136, … 1961]","[-0.117188, -0.65625, … -7.140625]","[0.0546875, 5.2109375, … 3.328125]","[[18342, 17252, … 17881], [18869, 17568, … 18350], … [18495, 18155, … 18521]]",16,"""micro"""
2023-03-06 22:59:01,"[1, 42, … 63]","[0, 0, … 0]",9.0,"[0.12932, 5.252098, … 7.878131]","[6140, 1149, … 1958]","[-0.117188, -0.65625, … -7.140625]","[0.0546875, 5.2109375, … 3.328125]","[[18114, 19102, … 18196], [18289, 19046, … 18045], … [18701, 19002, … 17740]]",16,"""micro"""
